In [258]:
import pandas as pd
import math
from datetime import datetime
import random
from IPython.display import display

In [359]:
LOCAL_SCHEDULE_FILEPATH = "F24-Schedule-ECE (3).csv"

MAX_ROOMS_ALLOWED = 2
MAX_NUM_COURSES = 3

TIME_SLOT_HOURS = range(8, 11)
MONDAY_TIMESLOTS = [f"Mon_{h:02d}:00" for h in TIME_SLOT_HOURS]

# AWS
REGION = "us-east-1"
session = boto3.Session(region_name=REGION)
sts = session.client("sts")
account_id = sts.get_caller_identity()["Account"]
s3_bucket = f"amazon-braket-{REGION}-{account_id}"

print("AWS Account:", account_id)
print("Braket Bucket:", s3_bucket)


AWS Account: 381305464630
Braket Bucket: amazon-braket-us-east-1-381305464630


In [360]:
def calculate_duration(begin_str, end_str):
    fmt = "%I:%M%p"
    try:
        if pd.isna(begin_str) or pd.isna(end_str):
            return 1
        
        t0 = datetime.strptime(begin_str.strip(), fmt)
        t1 = datetime.strptime(end_str.strip(), fmt)

        diff = (t1 - t0).total_seconds() / 3600

        # If exactly 1 hour, return 1
        if abs(diff - 1.0) < 0.01:
            return 1

        return max(1, math.ceil(diff))
    except:
        return 1


In [550]:
def load_and_force_monday(filepath):

    df = pd.read_csv(filepath)
    df.columns = [c.strip() for c in df.columns]

    # convert course num to int
    df["COURSE_NUM"] = pd.to_numeric(df["COURSE"], errors="coerce")
    df["SEC"] = df["SEC"].astype(str).str.strip()

    # EXACT courses we want
    target = [
        (18021, "A1"),   # PCB Fabrication
        (18095, "A"),    # Getting Started in Electronics
        (18100, "Lec1"), # Intro to ECE
    ]

    mask = False
    for cid, sec in target:
        mask = mask | ((df["COURSE_NUM"] == cid) & (df["SEC"] == sec))

    df = df[mask].copy().head(3)

    # Build MeetingID
    df["MeetingID"] = df["COURSE"].astype(str) + "-" + df["SEC"]

    # Instructor list
    df["INSTRUCTORS"] = (
        df["INSTRUCTOR"]
        .fillna("")
        .apply(lambda x: [t.strip() for t in x.split(",") if t.strip() != ""])
    )

    # Duration slots (raw)
    df["DurationSlots"] = df.apply(
        lambda r: calculate_duration(r["BEGIN"], r["END"]), axis=1
    )

    # Extract meetings
    meetings = df["MeetingID"].tolist()

    # REAL rooms: ANS B10, HH A104, SH 105
    all_rooms = df["BLDG/ROOM"].tolist()
    seen = set()
    rooms = []
    for r in all_rooms:
        if r not in seen:
            seen.add(r)
            rooms.append(r)
    rooms = rooms[:MAX_ROOMS_ALLOWED]

    meeting_instructors = df.set_index("MeetingID")["INSTRUCTORS"].to_dict()
    meeting_duration    = df.set_index("MeetingID")["DurationSlots"].to_dict()

    # -----------------------------------------------------
    # FORCE CUSTOM DURATIONS
    # -----------------------------------------------------
    # Treat 18095-A as a 1-hour class
    if "18095-A" in meeting_duration:
        meeting_duration["18095-A"] = 1

    # # optional: keep others correct
    # if "18021-A1" in meeting_duration:
    #     meeting_duration["18021-A1"] = 2
    # if "18100-Lec1" in meeting_duration:
    #     meeting_duration["18100-Lec1"] = 2

    # synthetic enrollment & capacity
    enrollment = {
        '18021-A1': 28,
        '18095-A': 20,
        '18100-Lec1': 45
    }
    
    capacity = {
        'ANS B10': 30,
        'HH A104': 50
    }

    print("\nLoaded EXACT 3 courses:")
    display(df[["MeetingID","COURSE TITLE","BEGIN","END","BLDG/ROOM","INSTRUCTOR"]])

    print("\nRooms:", rooms)
    print("Monday slots:", MONDAY_TIMESLOTS)
    print("Durations:", meeting_duration)
    print("Enrollment:", enrollment)
    print("Capacity:", capacity)

    return meetings, rooms, meeting_instructors, meeting_duration, enrollment, capacity


In [511]:
(meetings, rooms, meeting_instructors,
 duration_map, enrollment, capacity) = load_and_force_monday(LOCAL_SCHEDULE_FILEPATH)


Loaded EXACT 3 courses:


,MeetingID,COURSE TITLE,BEGIN,END,BLDG/ROOM,INSTRUCTOR
0,18021-A1,Introduction to Printed Circuit Boards Fabrica...,08:00AM,09:20AM,ANS B10,"Bain, James"
3,18095-A,Getting Started in Electronics: An Experientia...,11:00AM,12:20PM,HH A104,"Zajdel, Tom; Youssfi, Ziad"
5,18100-Lec1,Introduction to Electrical and Computer Engine...,02:00PM,03:20PM,SH 105,"Zhu, Jian-Gang"



Rooms: ['ANS B10', 'HH A104']
Monday slots: ['Mon_08:00', 'Mon_09:00', 'Mon_10:00']
Durations: {'18021-A1': 2, '18095-A': 1, '18100-Lec1': 2}
Enrollment: {'18021-A1': 28, '18095-A': 20, '18100-Lec1': 45}
Capacity: {'ANS B10': 30, 'HH A104': 50}


In [512]:
valid_vars = {}
vid = 0

for m in meetings:                      # all 3 courses exist here
    dur = duration_map[m]

    for r in rooms:
        for i, ts in enumerate(MONDAY_TIMESLOTS):

            # ensure meeting fits
            if i + dur - 1 >= len(MONDAY_TIMESLOTS):
                continue

            # this is a valid START
            valid_vars[(m, r, ts)] = vid
            vid += 1

print("Valid qubits:", len(valid_vars))
print("\nSample valid assignments:")
for k,v in list(valid_vars.items()):
    print(k,v)


Valid qubits: 14

Sample valid assignments:
('18021-A1', 'ANS B10', 'Mon_08:00') 0
('18021-A1', 'ANS B10', 'Mon_09:00') 1
('18021-A1', 'HH A104', 'Mon_08:00') 2
('18021-A1', 'HH A104', 'Mon_09:00') 3
('18095-A', 'ANS B10', 'Mon_08:00') 4
('18095-A', 'ANS B10', 'Mon_09:00') 5
('18095-A', 'ANS B10', 'Mon_10:00') 6
('18095-A', 'HH A104', 'Mon_08:00') 7
('18095-A', 'HH A104', 'Mon_09:00') 8
('18095-A', 'HH A104', 'Mon_10:00') 9
('18100-Lec1', 'ANS B10', 'Mon_08:00') 10
('18100-Lec1', 'ANS B10', 'Mon_09:00') 11
('18100-Lec1', 'HH A104', 'Mon_08:00') 12
('18100-Lec1', 'HH A104', 'Mon_09:00') 13


In [513]:
Q = {}
def addQ(i, j, v):
    """Safely add QUBO coefficient."""
    Q[(i, j)] = Q.get((i, j), 0.0) + v

def qubits_for_meeting(m):
    return [qid for (mm, r, ts), qid in valid_vars.items() if mm == m]

def qubits_for_room_timeslot(room, ts):
    return [qid for (m, r, t), qid in valid_vars.items() if r == room and t == ts]

def qubits_for_instructor_at_timeslot(inst, ts):
    return [
        qid for (m, r, t), qid in valid_vars.items()
        if t == ts and m in inst_map.get(inst, [])
    ]

from itertools import combinations

# --------------------
# Helper: all START variables for a meeting
# --------------------
def start_qubits_for_meeting(m):
    """Return only the qubits where the meeting STARTS."""
    starts = []
    for (mm, r, ts), qid in valid_vars.items():
        if mm != m:
            continue
        # check if ts is a valid start for duration
        start_idx = MONDAY_TIMESLOTS.index(ts)
        if start_idx + duration_map[m] - 1 < len(MONDAY_TIMESLOTS):
            starts.append(qid)
    return starts

# ================================================================
# L1 — Exactly one START per meeting (HARD)
# ================================================================
def add_L1(L1=8000):
    for m in meetings:
        ids = start_qubits_for_meeting(m)

        # Reward 1 start
        for i in ids:
            addQ(i, i, -L1)

        # Penalize 2 starts
        for i, j in combinations(ids, 2):
            addQ(i, j, 2 * L1)

# ================================================================
# L2 — One meeting per room per timeslot (HARD)
# ================================================================
def add_L2(L2=6000):
    for r in rooms:
        for ts in MONDAY_TIMESLOTS:
            ids = qubits_for_room_timeslot(r, ts)
            for i, j in combinations(ids, 2):
                addQ(i, j, L2)

# ================================================================
# L3 — Instructor conflict (HARD-ish)
# ================================================================
def add_L3(L3=4000):
    for inst, mlist in inst_map.items():
        if len(mlist) <= 1:
            continue
        for ts in MONDAY_TIMESLOTS:
            ids = qubits_for_instructor_at_timeslot(inst, ts)
            for i, j in combinations(ids, 2):
                addQ(i, j, L3)

# ================================================================
# L4 — Capacity penalty (SOFT)
# ================================================================
def add_L4(L4=20):
    for (m, r, ts), qid in valid_vars.items():
        if enrollment[m] > capacity[r]:
            addQ(qid, qid, L4)

# ================================================================
# L5 — Meeting must stay in ONE room across entire duration (HARD)
# ================================================================
def add_L5(L5=8000):
    for m in meetings:

        # group qubits by room
        room_to_qids = {}

        for (mm, r, ts), qid in valid_vars.items():
            if mm == m:
                room_to_qids.setdefault(r, []).append(qid)

        rooms_list = list(room_to_qids.keys())

        # penalize if ANY qubits of meeting are in two different rooms
        for i in range(len(rooms_list)):
            for j in range(i+1, len(rooms_list)):
                r1 = rooms_list[i]
                r2 = rooms_list[j]
                for q1 in room_to_qids[r1]:
                    for q2 in room_to_qids[r2]:
                        addQ(q1, q2, L5)

# ================================================================
# L6 — Continuity reward (SOFT)
# ================================================================
def add_L6(L6=200):
    for m in meetings:
        dur = duration_map[m]

        for r in rooms:
            for i, ts in enumerate(MONDAY_TIMESLOTS):

                # only valid starts
                if (m, r, ts) not in valid_vars:
                    continue

                start_qid = valid_vars[(m, r, ts)]
                start_idx = i

                # reward consecutive timeslots
                for k in range(1, dur):
                    if start_idx + k >= len(MONDAY_TIMESLOTS):
                        break

                    next_ts = MONDAY_TIMESLOTS[start_idx + k]
                    if (m, r, next_ts) in valid_vars:
                        next_qid = valid_vars[(m, r, next_ts)]
                        addQ(start_qid, next_qid, -L6)
                        addQ(next_qid, start_qid, -L6)


In [514]:
def build_qubo():
    global Q
    Q = {}  # reset

    add_L1()
    add_L2()
    add_L3()
    add_L4()
    add_L5()
    add_L6()

    print("QUBO built! Terms:", len(Q))
    return Q


# Call builder
Q = build_qubo()

QUBO built! Terms: 57


In [551]:
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
job_name = f"qaoa-scheduler-{timestamp}"
prefix = f"jobs/{job_name}/input"

s3 = boto3.client("s3")

def upload_json(obj, name):
    key = f"{prefix}/{name}"
    s3.put_object(Body=json.dumps(obj), Bucket=s3_bucket, Key=key)
    print("Uploaded:", key)
    return f"s3://{s3_bucket}/{key}"

qubo_s3 = upload_json({str(k):v for k,v in Q.items()}, "qubo.json")
var_s3  = upload_json({str(k):v for k,v in valid_vars.items()}, "var_map.json")


Uploaded: jobs/qaoa-scheduler-20251201-164507/input/qubo.json
Uploaded: jobs/qaoa-scheduler-20251201-164507/input/var_map.json


In [572]:
from braket.aws import AwsQuantumJob
from braket.jobs.config import OutputDataConfig, CheckpointConfig

job_name = f"qaoa-scheduler-{timestamp}"

job = AwsQuantumJob.create(
    job_name=job_name,
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="src",
    entry_point="src.hybrid_runner:run",
    input_data={"input": f"s3://{s3_bucket}/{prefix}/"},
    hyperparameters={"p":4, "shots":2000},
    output_data_config=OutputDataConfig(
        s3Path=f"s3://{s3_bucket}/jobs/{job_name}/output"
    ),
    checkpoint_config=CheckpointConfig(
        s3Uri=f"s3://{s3_bucket}/jobs/{job_name}/checkpoints"
    ),
    wait_until_complete=False,
)

print("Hybrid Job Submitted!")
print("Job ARN:", job.arn)


Hybrid Job Submitted!
Job ARN: arn:aws:braket:us-east-1:381305464630:job/702b7b50-ab25-437c-b4a6-bdc508fd6e6f


In [568]:
# import boto3

# bucket = "amazon-braket-us-east-1-381305464630"
# s3 = boto3.client("s3")

# resp = s3.list_objects_v2(
#     Bucket=bucket,
#     Prefix="jobs/schedule_qaoa/",
#     Delimiter="/"
# )

# prefixes = [p["Prefix"] for p in resp.get("CommonPrefixes", [])]
# latest_prefix = sorted(prefixes)[-1]

# print("Latest Monday-only job prefix:", latest_prefix)


In [569]:
# resp2 = s3.list_objects_v2(
#     Bucket=bucket,
#     Prefix=latest_prefix,
# )

# print("\n=== OBJECTS FOUND IN LATEST JOB ===\n")
# for obj in resp2.get("Contents", []):
#     print(obj["Key"])


In [415]:
# from braket.aws import AwsQuantumJob

# job = AwsQuantumJob("arn:aws:braket:us-east-1:381305464630:job/728c0701-f59e-4446-a46b-e556a050fe11")
# print(job.state())

COMPLETED


In [570]:
print(job.name)


qaoa-scheduler-20251201-164507


In [571]:
job_prefix = f"jobs/{job.name}/"
print("Correct job prefix:", job_prefix)


Correct job prefix: jobs/qaoa-scheduler-20251201-164507/


In [556]:
resp = s3.list_objects_v2(
    Bucket=bucket,
    Prefix=job_prefix,
)

for obj in resp.get("Contents", []):
    print(obj["Key"])


jobs/qaoa-scheduler-20251201-164507/1764607513647/script/source.tar.gz
jobs/qaoa-scheduler-20251201-164507/input/qubo.json
jobs/qaoa-scheduler-20251201-164507/input/var_map.json
jobs/qaoa-scheduler-20251201-164507/output/output/model.tar.gz
jobs/qaoa-scheduler-20251201-164507/tasks/eedb0584-fdf6-4afc-933b-25f687761258/results.json


In [557]:
s3.download_file(
    bucket,
    "jobs/qaoa-scheduler-20251201-164507/output/output/model.tar.gz",
    "model.tar.gz"
)


In [558]:
import tarfile
with tarfile.open("model.tar.gz") as tar:
    tar.extractall("model_output")


In [559]:
import boto3

bucket = "amazon-braket-us-east-1-381305464630"
prefix = "jobs/qaoa-scheduler-20251201-164507/output/output/model.tar.gz"

s3 = boto3.client("s3")

s3.download_file(
    bucket,
    prefix,
    "model.tar.gz"
)

print("Downloaded model.tar.gz")


Downloaded model.tar.gz


In [560]:
print("s3_bucket =", s3_bucket)
print("correct bucket =", "amazon-braket-us-east-1-381305464630")


s3_bucket = amazon-braket-us-east-1-381305464630
correct bucket = amazon-braket-us-east-1-381305464630


In [471]:
# s3.download_file(
#     s3_bucket,
#     f"jobs/{job_name}/output/model.tar.gz",
#     "model.tar.gz"
# )

# import tarfile
# with tarfile.open("model.tar.gz") as tar:
#     tar.extractall("model_output")


ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [451]:
# s3.download_file(bucket,
#                  "jobs/schedule_qaoa/20251201-095730/input/var_map_v4.json",
#                  "var_map_v4.json")

# s3.download_file(bucket,
#                  "jobs/schedule_qaoa/20251201-095730/input/qubo_v4.json",
#                  "qubo_v4.json")

# print("Downloaded var_map_v4.json and qubo_v4.json")


Downloaded var_map_v4.json and qubo_v4.json


In [561]:
import tarfile
with tarfile.open("model.tar.gz") as tar:
    tar.extractall("model_output")


In [562]:
resp = s3.list_objects_v2(
    Bucket=bucket,
    Prefix="jobs/qaoa-scheduler-20251201-164507/"
)

for obj in resp.get("Contents", []):
    print(obj["Key"])


jobs/qaoa-scheduler-20251201-164507/1764607513647/script/source.tar.gz
jobs/qaoa-scheduler-20251201-164507/input/qubo.json
jobs/qaoa-scheduler-20251201-164507/input/var_map.json
jobs/qaoa-scheduler-20251201-164507/output/output/model.tar.gz
jobs/qaoa-scheduler-20251201-164507/tasks/eedb0584-fdf6-4afc-933b-25f687761258/results.json


In [563]:
bucket = "amazon-braket-us-east-1-381305464630"

# download model archive
s3.download_file(
    bucket,
    "jobs/qaoa-scheduler-20251201-164507/output/output/model.tar.gz",
    "model.tar.gz"
)


In [564]:
import shutil, tarfile
shutil.rmtree("model_output", ignore_errors=True)

with tarfile.open("model.tar.gz") as tar:
    tar.extractall("model_output")


In [565]:
s3.download_file(
    bucket,
    "jobs/qaoa-scheduler-20251201-164507/input/var_map.json",
    "var_map.json"
)


In [566]:
import json

# ============================================================
# 1. LOAD RESULTS FROM model_output/
# ============================================================

with open("model_output/results.json") as f:
    results = json.load(f)

# TN1 stores counts here:
counts = results["dataDictionary"]["counts"]
print("Loaded counts:", len(counts))

with open("var_map.json") as f:
    raw_var_map = json.load(f)   # keys = "(m,r,ts)", values = qid integers

# Reverse mapping: qid → tuple (m,r,ts)
var_map_rev = {v: eval(k) for k, v in raw_var_map.items()}

print("Var map size:", len(var_map_rev))


# ============================================================
# 2. PICK BEST BITSTRING FROM COUNTS
# ============================================================

bitstring = max(counts, key=counts.get)
print("\nRaw best bitstring:", bitstring)

# Fix TN1 padding
nq = len(var_map_rev)
bitstring = bitstring[:nq]

print("Trimmed bitstring:", bitstring)

selected_qids = [i for i, b in enumerate(bitstring) if b == "1"]
print("Selected QUBO variables:", selected_qids)


# ============================================================
# 3. DECODE SELECTED ASSIGNMENTS
# ============================================================

assignments = [var_map_rev[qid] for qid in selected_qids]
print("\nDecoded assignments:")
for a in assignments:
    print(a)


# ============================================================
# 4. CLEAN AND PRINT SCHEDULE (WITH PROFESSORS)
# ============================================================

def clean_and_pretty_print(assignments, duration_map, enrollment, capacity, meeting_instructors):
    print("\n=========== CLEAN FINAL QAOA SCHEDULE (MONDAY ONLY) ===========\n")

    # Filter rooms
    assignments = [(m, r, ts) for (m, r, ts) in assignments if r in capacity]

    # Filter phantom meetings
    valid_meetings = set(duration_map.keys())
    assignments = [(m, r, ts) for (m, r, ts) in assignments if m in valid_meetings]

    meeting_candidates = {}
    for m, r, ts in assignments:
        meeting_candidates.setdefault(m, set()).add(r)
    
    meeting_room = {}
    
    used_rooms = set()
    
    for m, rooms in meeting_candidates.items():
        # Choose a room that fits enrollment and is not used
        possible = [r for r in rooms 
                    if capacity[r] >= enrollment[m] and r not in used_rooms]
    
        # If all candidate rooms are taken or too small,
        # fall back to ANY room with enough capacity
        if not possible:
            possible = [r for r in capacity 
                        if capacity[r] >= enrollment[m] and r not in used_rooms]
    
        # If still none, put it anywhere (worst case fallback)
        if not possible:
            possible = list(capacity.keys())
    
        chosen = possible[0]
        meeting_room[m] = chosen
        used_rooms.add(chosen)

    cleaned = [(m, meeting_room[m], ts) for (m, r, ts) in assignments]
    cleaned = list(set(cleaned))

    # Build timeline
    schedule = {}

    for m, r, ts in cleaned:
        dur = duration_map[m]
        start_hr = int(ts.split("_")[1].split(":")[0])

        schedule.setdefault(ts, []).append((m, r, "Start"))

        for k in range(1, dur):
            cont_ts = f"Mon_{start_hr+k:02d}:00"
            schedule.setdefault(cont_ts, []).append((m, r, "Cont."))

    # Print schedule
    for hour in [8, 9, 10, 11]:
        ts = f"Mon_{hour:02d}:00"
        print(ts + ":")

        if ts not in schedule:
            print("  (empty)\n")
            continue

        for m, r, tag in schedule[ts]:

            # GET PROFESSORS FOR MEETING
            profs = ", ".join(meeting_instructors.get(m, ["Unknown"]))

            print(f"  ({tag}) {m} | Prof(s): {profs} | Room {r} | "
                  f"{duration_map[m]} hr | "
                  f"cap={capacity[r]}, enroll={enrollment[m]}")
        print()


# ============================================================
# 5. PRINT
# ============================================================

clean_and_pretty_print(
    assignments,
    duration_map,
    enrollment,
    capacity,
    meeting_instructors  # <-- NEW
)


Loaded counts: 2000
Var map size: 14

Raw best bitstring: 011011111000110001101101
Trimmed bitstring: 01101111100011
Selected QUBO variables: [1, 2, 4, 5, 6, 7, 8, 12, 13]

Decoded assignments:
('18021-A1', 'ANS B10', 'Mon_09:00')
('18021-A1', 'HH A104', 'Mon_08:00')
('18095-A', 'ANS B10', 'Mon_08:00')
('18095-A', 'ANS B10', 'Mon_09:00')
('18095-A', 'ANS B10', 'Mon_10:00')
('18095-A', 'HH A104', 'Mon_08:00')
('18095-A', 'HH A104', 'Mon_09:00')
('18100-Lec1', 'HH A104', 'Mon_08:00')
('18100-Lec1', 'HH A104', 'Mon_09:00')

=========== CLEAN FINAL QAOA SCHEDULE (MONDAY ONLY) ===========

Mon_08:00:
  (Start) 18100-Lec1 | Prof(s): Zhu, Jian-Gang | Room ANS B10 | 2 hr | cap=30, enroll=45
  (Start) 18021-A1 | Prof(s): Bain, James | Room ANS B10 | 2 hr | cap=30, enroll=28
  (Start) 18095-A | Prof(s): Zajdel, Tom; Youssfi, Ziad | Room HH A104 | 1 hr | cap=50, enroll=20

Mon_09:00:
  (Start) 18100-Lec1 | Prof(s): Zhu, Jian-Gang | Room ANS B10 | 2 hr | cap=30, enroll=45
  (Cont.) 18100-Lec1 | Pr

In [396]:
# %%writefile src/hybrid_runner.py
# # src/hybrid_runner.py

# import os
# import json
# import numpy as np
# from braket.circuits import Circuit
# from braket.aws import AwsDevice
# from braket.jobs import save_job_result


# # ============================================================
# # 1. LOAD QUBO (JSON FORMAT)
# # ============================================================

# def load_qubo_json(input_path):
#     """Load QUBO from qubo.json and var_map.json inside mounted directory."""
#     qubo_file = os.path.join(input_path, "qubo.json")
#     varmap_file = os.path.join(input_path, "var_map.json")

#     print("Looking for QUBO at:", qubo_file)
#     print("Looking for var_map at:", varmap_file)

#     if not os.path.exists(qubo_file):
#         raise FileNotFoundError(f"QUBO file missing: {qubo_file}")

#     if not os.path.exists(varmap_file):
#         raise FileNotFoundError(f"var_map.json missing: {varmap_file}")

#     with open(qubo_file, "r") as f:
#         raw = json.load(f)

#     with open(varmap_file, "r") as f:
#         var_map = json.load(f)

#     # Convert keys "(i, j)" → tuple(i, j)
#     Q = {}
#     for k, v in raw.items():
#         i, j = eval(k)
#         Q[(i, j)] = float(v)

#     return Q, var_map


# # ============================================================
# # 2. CONVERT QUBO TO ISING
# # ============================================================

# def qubo_to_ising(Q):
#     """Convert QUBO dict into Ising h, J."""
#     n = max(max(i, j) for (i, j) in Q.keys()) + 1
#     h = np.zeros(n)
#     J = np.zeros((n, n))

#     for (i, j), q in Q.items():
#         if i == j:
#             h[i] += q
#         else:
#             J[i, j] += q / 2
#             J[j, i] += q / 2

#     return h, J


# # ============================================================
# # 3. ONE QAOA LAYER
# # ============================================================

# def build_qaoa_layer(circ: Circuit, h, J, gamma, beta):
#     """Append a QAOA layer."""
#     n = len(h)

#     # Cost Hamiltonian
#     for i in range(n):
#         if h[i] != 0:
#             circ.rz(i, 2 * gamma * h[i])

#     for i in range(n):
#         for j in range(i + 1, n):
#             if J[i, j] != 0:
#                 circ.cnot(i, j)
#                 circ.rz(j, 2 * gamma * J[i, j])
#                 circ.cnot(i, j)

#     # Mixer
#     for i in range(n):
#         circ.rx(i, 2 * beta)

#     return circ


# # ============================================================
# # 4. FULL QAOA CIRCUIT
# # ============================================================

# def build_qaoa_circuit(p, h, J, gammas, betas):
#     circ = Circuit()
#     n = len(h)

#     circ.h(range(n))  # uniform superposition

#     for layer in range(p):
#         build_qaoa_layer(circ, h, J, gammas[layer], betas[layer])

#     for i in range(n):
#         circ.measure(i)

#     return circ


# # ============================================================
# # 5. HYBRID JOB ENTRYPOINT
# # ============================================================

# def run(**kwargs):
#     print("=== Amazon Braket Hybrid Job Started ===")

#     # Hyperparameters
#     p = int(kwargs.get("p", 1))
#     shots = int(kwargs.get("shots", 500))

#     gammas = kwargs.get("gammas") or [0.1] * p
#     betas  = kwargs.get("betas") or [0.2] * p

#     print(f"Hyperparameters: p={p}, shots={shots}")
#     print("gammas:", gammas)
#     print("betas:", betas)

#     # ============================================================
#     # FIXED: Correct input directory for hybrid jobs
#     # ============================================================
#     # Amazon Braket ALWAYS mounts input data here:
#     #     /opt/braket/input/data/<channel>/
#     base_dir = "/opt/braket/input/data/input"
#     print("Resolved input path:", base_dir)

#     # ============================================================
#     # Load QUBO
#     # ============================================================
#     Q, var_map = load_qubo_json(base_dir)
#     print("Loaded QUBO terms:", len(Q))

#     # Convert QUBO → Ising
#     h, J = qubo_to_ising(Q)
#     n = len(h)
#     print("Number of qubits:", n)

#     if n > 50:
#         raise ValueError("TN1 cannot run >50 qubits. Reduce problem size.")

#     # ============================================================
#     # Build p-layer QAOA circuit
#     # ============================================================
#     print("Building QAOA circuit...")
#     circ = build_qaoa_circuit(p, h, J, gammas, betas)

#     # ============================================================
#     # Run circuit on TN1 or local simulator
#     # ============================================================
#     device_arn = os.environ["AMZN_BRAKET_DEVICE_ARN"]
#     print("Running on device:", device_arn)

#     device = AwsDevice(device_arn)
#     task = device.run(circ, shots=shots)
#     result = task.result()

#     # Convert bitstrings to JSON-serializable form
#     counts = {str(bits): int(cnt) for bits, cnt in result.measurement_counts.items()}

#     print("Counts:", counts)

#     # ============================================================
#     # Output final results
#     # ============================================================
#     save_job_result({
#         "counts": counts,
#         "qubo_terms": len(Q),
#         "n_qubits": n,
#         "p": p,
#         "gammas": gammas,
#         "betas": betas,
#         "shots": shots
#     })

#     print("=== Hybrid Job DONE ===")

#     return {
#         "counts": counts,
#         "qubo_terms": len(Q),
#         "n_qubits": n,
#         "p": p,
#         "gammas": gammas,
#         "betas": betas,
#         "shots": shots
#     }

Overwriting src/hybrid_runner.py


In [239]:
import boto3, tarfile, json, os

bucket = s3_bucket = f"amazon-braket-us-east-1-{account_id}"
s3 = boto3.client("s3")

# 1. List all qaoa-scheduler jobs
resp = s3.list_objects_v2(
    Bucket=bucket,
    Prefix="jobs/qaoa-scheduler",
    Delimiter="/"
)

prefixes = [x["Prefix"] for x in resp.get("CommonPrefixes", [])]
latest = sorted(prefixes)[-1]

print("Newest prefix:", latest)

# 2. Find model.tar.gz inside output/
resp2 = s3.list_objects_v2(
    Bucket=bucket,
    Prefix=latest + "output/"
)

# find model.tar.gz
model_key = None
for obj in resp2.get("Contents", []):
    if obj["Key"].endswith("model.tar.gz"):
        model_key = obj["Key"]

if not model_key:
    raise FileNotFoundError("model.tar.gz not found in latest job output.")

print("Found model:", model_key)

# 3. Download
local_tar = "model_latest.tar.gz"
s3.download_file(bucket, model_key, local_tar)
print("Downloaded:", local_tar)

# 4. Extract
extract_dir = "model_output"
with tarfile.open(local_tar, "r:gz") as tar:
    tar.extractall(extract_dir)

print("Extracted to:", extract_dir)


Newest prefix: jobs/qaoa-scheduler-20251201-085226/
Found model: jobs/qaoa-scheduler-20251201-085226/output/output/model.tar.gz
Downloaded: model_latest.tar.gz
Extracted to: model_output


In [243]:
import json
import ast

with open("qubo_v1.json") as f:
    raw_qubo = json.load(f)

with open("var_map_v1.json") as f:
    raw_varmap = json.load(f)

Q = {ast.literal_eval(k): float(v) for (k, v) in raw_qubo.items()}
var_map = {ast.literal_eval(k): int(v) for (k, v) in raw_varmap.items()}

print("Loaded QUBO terms:", len(Q))
print("Loaded var_map entries:", len(var_map))


Loaded QUBO terms: 57
Loaded var_map entries: 14


In [554]:
from braket.circuits import Circuit
import json
import numpy as np

# -------------------------------------------------------------
# LOAD DOWNLOADED INPUT FILES FROM S3 JOB
# -------------------------------------------------------------
with open("qubo.json", "r") as f:
    raw_Q = json.load(f)

with open("var_map.json", "r") as f:
    var_map = json.load(f)

Q = {eval(k): float(v) for k, v in raw_Q.items()}

print("Loaded QUBO terms:", len(Q))
print("Loaded var_map entries:", len(var_map))


# -------------------------------------------------------------
# QUBO → ISING
# -------------------------------------------------------------
def qubo_to_ising(Q):
    n = max(max(i, j) for (i, j) in Q.keys()) + 1
    h = np.zeros(n)
    J = np.zeros((n, n))

    for (i, j), q in Q.items():
        if i == j:
            h[i] += q
        else:
            J[i, j] += q / 2
            J[j, i] += q / 2

    return h, J

h, J = qubo_to_ising(Q)
n_qubits = len(h)

print("Number of qubits:", n_qubits)


# -------------------------------------------------------------
# QAOA LAYER
# -------------------------------------------------------------
def build_qaoa_layer(circ, h, J, gamma, beta):
    n = len(h)

    # Cost: linear terms
    for i in range(n):
        if h[i] != 0:
            circ.rz(i, 2 * gamma * h[i])

    # Cost: quadratic terms
    for i in range(n):
        for j in range(i + 1, n):
            if J[i, j] != 0:
                circ.cnot(i, j)
                circ.rz(j, 2 * gamma * J[i, j])
                circ.cnot(i, j)

    # Mixer
    for i in range(n):
        circ.rx(i, 2 * beta)

    return circ


# -------------------------------------------------------------
# BUILD FULL CIRCUIT (NO MEASUREMENT FOR CLEAN VISUAL)
# -------------------------------------------------------------
def build_qaoa_circuit(p, h, J, gammas, betas):
    circ = Circuit()
    circ.h(range(len(h)))  # initial superposition

    for layer in range(p):
        build_qaoa_layer(circ, h, J, gammas[layer], betas[layer])

    return circ


# -------------------------------------------------------------
# SELECT DEPTH + PRETTY CIRCUIT
# -------------------------------------------------------------
p = 2   # or try 1, 2, or 3
gammas = [0.3] * p
betas  = [0.4] * p

circ = build_qaoa_circuit(p, h, J, gammas, betas)

print("\n=== QAOA Circuit (p =", p, ") ===\n")
print(circ)

# Jupyter pretty view
print(circ.diagram())



Loaded QUBO terms: 148
Loaded var_map entries: 14
Number of qubits: 24

=== QAOA Circuit (p = 2 ) ===

T   : │  0  │      1      │  2  │      3      │  4  │  5  │     6      │  7  │     8     │      9      │    10     │    11     │     12     │    13     │       14        │     15     │       16        │       17        │     18     │       19        │          20           │     21      │          22           │              23              │     24      │          25           │                 26                 │     27      │             28              │                 29                  │     30     │             31              │                    32                    │     33      │                34                 │                 35                  │        36         │             37              │                 38                 │        39         │             40              │              41               │           42           │          43           │    

In [573]:
def qaoa_schematic(circ, p):
    """
    Produce an AWS-style simplified circuit diagram for QAOA:
    Each qubit shows:  H ─ UC ─ UB ─ UC ─ UB ─ M
    instead of thousands of rz/cnot gates.
    """

    n = circ.qubit_count

    print("\n==================== QAOA Schematic Diagram ====================\n")
    # Header
    header = "       "
    for layer in range(p):
        header += f"── UC(γ{layer+1}) ── UB(β{layer+1}) "
    header += "── Measure"
    print(header)
    print()

    # Qubits
    for q in range(n):
        line = f"q{q:<3}: H "
        for layer in range(p):
            line += "── UC ── UB "
        line += "── M"
        print(line)

    print("\n================================================================\n")


In [574]:
qaoa_schematic(circ, p)



==================== QAOA Schematic Diagram ====================

       ── UC(γ1) ── UB(β1) ── UC(γ2) ── UB(β2) ── Measure

q0  : H ── UC ── UB ── UC ── UB ── M
q1  : H ── UC ── UB ── UC ── UB ── M
q2  : H ── UC ── UB ── UC ── UB ── M
q3  : H ── UC ── UB ── UC ── UB ── M
q4  : H ── UC ── UB ── UC ── UB ── M
q5  : H ── UC ── UB ── UC ── UB ── M
q6  : H ── UC ── UB ── UC ── UB ── M
q7  : H ── UC ── UB ── UC ── UB ── M
q8  : H ── UC ── UB ── UC ── UB ── M
q9  : H ── UC ── UB ── UC ── UB ── M
q10 : H ── UC ── UB ── UC ── UB ── M
q11 : H ── UC ── UB ── UC ── UB ── M
q12 : H ── UC ── UB ── UC ── UB ── M
q13 : H ── UC ── UB ── UC ── UB ── M
q14 : H ── UC ── UB ── UC ── UB ── M
q15 : H ── UC ── UB ── UC ── UB ── M
q16 : H ── UC ── UB ── UC ── UB ── M
q17 : H ── UC ── UB ── UC ── UB ── M
q18 : H ── UC ── UB ── UC ── UB ── M
q19 : H ── UC ── UB ── UC ── UB ── M
q20 : H ── UC ── UB ── UC ── UB ── M
q21 : H ── UC ── UB ── UC ── UB ── M
q22 : H ── UC ── UB ── UC ── UB ── M
q23 : H ── UC ── UB ── 

In [576]:
from braket.circuits import Circuit

# A small demo QUBO for presentation (4 qubits)
# This does NOT need to be your real QUBO — it's just for visualization.
h_demo = [0.6, -1.1, 0.3, -0.5]
J_demo = {
    (0, 1): 0.8,
    (1, 2): -0.4,
    (2, 3): 0.7,
}

# ---- Build a small QAOA circuit with p=1 ----
def qaoa_small_demo(h, J, gamma=0.3, beta=0.4):
    n = len(h)
    circ = Circuit()
    
    # Initial superposition
    circ.h(range(n))
    
    # Cost Hamiltonian
    for i in range(n):
        circ.rz(i, 2 * gamma * h[i])
    for (i, j), val in J.items():
        circ.cnot(i, j)
        circ.rz(j, 2 * gamma * val)
        circ.cnot(i, j)
    
    # Mixer
    for i in range(n):
        circ.rx(i, 2 * beta)
    
    return circ

circ_demo = qaoa_small_demo(h_demo, J_demo)

# ---- Print clean ASCII diagram like AWS documentation ----
print(circ_demo)


T  : │  0  │      1      │  2  │     3      │  4  │     5      │      6      │  7  │     8      │     9      │ 10  │     11     │
      ┌───┐ ┌──────────┐                           ┌──────────┐                                                                  
q0 : ─┤ H ├─┤ Rz(0.36) ├────●──────────────────●───┤ Rx(0.80) ├──────────────────────────────────────────────────────────────────
      └───┘ └──────────┘    │                  │   └──────────┘                                                                  
      ┌───┐ ┌───────────┐ ┌─┴─┐ ┌──────────┐ ┌─┴─┐                                  ┌──────────┐                                 
q1 : ─┤ H ├─┤ Rz(-0.66) ├─┤ X ├─┤ Rz(0.48) ├─┤ X ├──────●───────────────────────●───┤ Rx(0.80) ├─────────────────────────────────
      └───┘ └───────────┘ └───┘ └──────────┘ └───┘      │                       │   └──────────┘                                 
      ┌───┐ ┌──────────┐                              ┌─┴─┐     ┌───────────┐ ┌─┴─┐       